<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/de_comparisons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist


# TowerDB Comparison

In [24]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save,old_file,new_file, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    import re
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule

    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        #df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        dates_tw = ['first_active_sharing_arrangement_start_date','first_active_sharing_arrangement_end_date', 'date_of_equipment_removal']
        for date_col in dates_tw:
            lista = []
            df[date_col] = df[date_col].replace([' '], '')
            df[date_col] = df[date_col].fillna('')
            for i in df[date_col]:
                dates_format = re.match(r'\d{2}\/\d{2}\/\d{4}', str(i)) or re.match(r'\d{2}\-\d{2}\-\d{4}', str(i))
                not_date_format = not re.match(r'\d{2}\/\d{2}\/\d{4}', str(i)) or not re.match(r'\d{2}\-\d{2}\-\d{4}', str(i))
                if i !='' and not_date_format:
                    #print(towerdb[towerdb[date_col]==i][['identification - site key', date_col]])
                    lista.append(f'{i:%d/%m/%Y}')
                
                elif dates_format:
                    lista.append(i)
                else:
                    lista.append('')
            df[date_col] = lista
        ints = ['security class', 'postal code']
        for i in ints:
            df[i] = df[i].fillna(0)
            df[i] = df[i].replace('', 0)
            df[i] = list(map(int, df[i]))

        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW[index_col] = df_NEW[index_col].astype(str)
        df_NEW['sites'] = df_NEW[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_NEW = df_NEW.set_index('sites').fillna('')

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save}_{old_file} vs {new_file}.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ20000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
        
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ20000', header_style)
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')
    

bill_cols = ["Code",\
            "Categorization by Transmission Sys",\
            "Categorization by Transmission Sys (sub-cluster)",\
            "Categorization by Site Type","Categorization by Indoor / Outdoor Site (Antennas)",\
            "Categorization by Strategic Site",\
            "Categorization by Critical Site",\
            "Site Cluster",\
            "Strategic Site Bucket",\
            "Bts_sites",\
            "CriticalSite_Beyond_10",\
            "Sites As Metered Estimated",\
            "Active Sharing Arrangement",\
            "Indoor_Site_Any_Climate_Control",\
            "Outdoor_Site_With_Dc_Power",\
            "Rfai_Date",\
            "decommissioned site",\
            "Subsequent_Active_Sharing_Arrangement",\
            "First_Active_Sharing_Deployment_Type",\
            "First_Active_Sharing_Arrangement_Start_Date",\
            "First_Active_Sharing_Arrangement_End_Date",\
            'Site Status (TIMS)']
pathnew = '/content/TowerDB_Germany_20210731.csv'
pathold = '/content/TowerDB_Germany_20210630.csv'
to_save = '/content/DE_TW'
old_n = '20210630.csv'
new_n = '20210731.csv'

find_diffs_between_files(pathold, pathnew, 'Code', bill_cols, 'Site Status (TIMS)', to_save,old_n, new_n, 'mix','tw')


New Rows:     ['0861 W', '3207 O', '5331 B', '5866 O', '6214 S', '6282 O', '7963 H', 'B194 M']
Dropped Rows: ['1130 M', '2890 W', 'MRT112', 'MRT125', 'MRT126', 'MRT147']

Done.



Special Case - Excel File(August)

In [ ]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save,old_file,new_file, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    import re
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule

    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        #df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        dates_tw = ['first_active_sharing_arrangement_start_date','first_active_sharing_arrangement_end_date', 'date_of_equipment_removal']
        for date_col in dates_tw:
            lista = []
            df[date_col] = df[date_col].replace([' '], '')
            df[date_col] = df[date_col].fillna('')
            for i in df[date_col]:
                dates_format = re.match(r'\d{2}\/\d{2}\/\d{4}', str(i)) or re.match(r'\d{2}\-\d{2}\-\d{4}', str(i))
                not_date_format = not re.match(r'\d{2}\/\d{2}\/\d{4}', str(i)) or not re.match(r'\d{2}\-\d{2}\-\d{4}', str(i))
                if i !='' and not_date_format:
                    #print(towerdb[towerdb[date_col]==i][['identification - site key', date_col]])
                    lista.append(f'{i:%d/%m/%Y}')
                
                elif dates_format:
                    lista.append(i)
                else:
                    lista.append('')
            df[date_col] = lista
        ints = ['security class', 'postal code']
        for i in ints:
            df[i] = df[i].fillna(0)
            df[i] = df[i].replace('', 0)
            df[i] = list(map(int, df[i]))

        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = path_NEW
        df_NEW[index_col] = df_NEW[index_col].astype(str)
        df_NEW = df_NEW.drop_duplicates(subset=index_col)
        df_NEW['sites'] = df_NEW[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_NEW = df_NEW.set_index('sites').fillna('')

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save}_{old_file} vs {new_file}.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ20000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
        
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ20000', header_style)
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')
    
path_excel = '/content/Output - Skylon - TowerDB - Germany - Release6.3 20210726_Billing_ReducedScope_sharedwithCelfinet_v03 (1).xlsx'
#towerdb = pd.read_csv(path_towerdb,encoding='latin')
guideline = lower_str(['Code','Site Name','Macro Region','Region','Province','Municipality','Inhabitants','Address','Latitude','Longitude',\
             'Categorization by Transmission Sys','Categorization by Transmission Sys (sub-cluster)','Categorization by Site Type',\
             'Categorization by Indoor / Outdoor Site (Antennas)','Additional Site Type information','Categorization by Inhabitants',\
             'Rural/ Suburban/ Urban','Categorization by CONNECTIVITY','Technology VOD','POD ID','Energy Consumption current FY',\
             'Actual Energy Cost current FY','Infrastructure ready (existing)/ to be ready (new)','Infrastructure to be shared by',\
             'Counterpart','# of Lease Contracts','Current annual lease fees','Current energy lease fees','Current annual other fees',\
             'Total Annual Lease','(Average) residual duration','Maturity Cluster','ExCo rep. Avg Annual Lease costs',\
             'Total Energy Cost currrent FY (Energy provider + LL)','VOD (y/n)','Deutsche Telekom','Annual Fee per Tenant MNO1',\
             'Annual Energy Fee MNO1','Annual Maintenance Fee MNO1','Other Services Fee MNO1','Total Hosting Fee & Services MNO1',\
             'Residual duration MNO1','Maturity Clusters MNO1','Telefonica','Annual Fee per Tenant MNO2','Annual Energy Fee MNO2',\
             'Annual Maintenance Fee MNO2','Other Services Fee MNO2','Total Hosting Fee & Services MNO2','Residual duration MNO2',\
             'Maturity Clusters MNO2','E-Plus','Annual Fee per Tenant MNO3','Annual Energy Fee MNO3','Annual Maintenance Fee MNO3',\
             'Other Services Fee MNO3','Total Hosting Fee & Services','Residual duration MNO3','Maturity Clusters','# of OTMOs',\
             'Annual Fee from OTMOs','Annual Energy Fee from OTMOs','Annual Maintenance Fee OTMOs','Other Services Fee OTMOs',\
             'Total Hosting Fee & Services OTMOs','Average residual duration OTMOs','Maturity Clusters OTMOs','Total # of 3rd Party Tenants',\
             'Annual Fee from 3rd Party Tenants','Annual Energy Fee from 3rd Party Tenants','Annual Maintenance Fee from 3rd Party Tenants',\
             'Other Services Fee from 3rd Party Tenants','Total Hosting Fee & Services from 3rd Party Tenants',\
             'Weighted Average residual duration','Macro Cluster Tenancy','Macro Cluster Lease / Freeholds & Surface Right','Macro Cluster 1',\
             'Sites w/ at least a DDS (Lease Contract Type)','# of Tenants','Categorization by Tenant combination',\
             'Categorization by Type of Passive contracts','Categorization by Land/Surface ownership','In/Out',\
             'No of 3rd Party Tenants (PoP count)','Top cities classification','Type of Structure','Categorization by shared usage (contractual)',\
             'Categorization by Shared Usage (PoP count)','Security Class','Categorization by Sites with Single/Multiple Tower(s)',\
             'Categorization by Special Site/PLC','Categorization by Chimney-location','Categorization by Antenna Constraint',\
             'Categorization by 5G Roll-out Site','Categorization by Authority Site','Categorization by Transport-Sammler',\
             'Categorization by Strategic Site','Categorization by Dual Use Site','Categorization by Sublease Right','GBT Reserve Space','State',\
             'Postal code','Tower Height','Floor Space','Ground Register .1','Ground Register .2','Ground Register .3',\
             'Categorization by Critical Site','Site Cluster','infrastructure to be dismantled by','decommissioned site',\
             'Strategic Site Bucket','Bts_sites','CriticalSite_Beyond_10','Sites As Metered Estimated','Active Sharing Arrangement',\
             'Subsequent_Active_Sharing_Arrangement','First_Active_Sharing_Deployment_Type','First_Active_Sharing_Arrangement_Start_Date',\
             'First_Active_Sharing_Arrangement_End_Date','Indoor_Site_Any_Climate_Control','Outdoor_Site_With_Dc_Power','Rfai_Date',\
             'Cluster for non-enterprise DAS Sites','Site Status (TIMS)','Date_Of_Equipment_Removal'])

new = pd.read_excel(path_excel, sheet_name='TowerDB', skiprows=12)
new = new.iloc[:, 2:]
new.columns = lower_str(list(new.columns))

path_towerdb = '/content/TowerDB_Germany_20210731.csv'
towerdb = pd.read_csv(path_towerdb,encoding='latin')
towerdb.columns = lower_str(list(towerdb.columns))

columns_to_merge = ['code', 'annual maintenance fee mno3', 'categorization by critical site', 'e-plus', 'categorization by antenna constraint', 'ground register .3', 'criticalsite_beyond_10', 'energy consumption current fy', 'first_active_sharing_deployment_type', 'indoor_site_any_climate_control', 'total hosting fee & services', 'subsequent_active_sharing_arrangement', 'categorization by indoor / outdoor site (antennas)', 'outdoor_site_with_dc_power', 'additional site type information', 'annual fee per tenant mno3', 'total energy cost currrent fy (energy provider + ll)', 'rfai_date', 'cluster for non-enterprise das sites', 'residual duration mno3', 'other services fee mno3', 'actual energy cost current fy', 'annual energy fee mno3', 'ground register .2', 'first_active_sharing_arrangement_start_date', 'ground register .1', 'first_active_sharing_arrangement_end_date', 'maturity clusters']
df_merge = towerdb[columns_to_merge]
new = pd.merge(new, df_merge, how='left', on='code')

new = new[guideline]

dates_tw = ['first_active_sharing_arrangement_start_date','first_active_sharing_arrangement_end_date', 'date_of_equipment_removal']
for i in dates_tw:
    new[i] = [date_obj.strftime("%d/%m/%Y") if not pd.isnull(date_obj) else '' for date_obj in new[i]]

ints = ['actual energy cost current fy', 'counterpart', '# of lease contracts', 'current energy lease fees',\
        'total annual lease', 'deutsche telekom', 'annual energy fee mno1', 'total hosting fee & services mno1', 'telefonica',\
        'annual energy fee mno2', '# of otmos', 'annual energy fee from otmos', 'total hosting fee & services otmos',\
        'total # of 3rd party tenants', 'annual energy fee from 3rd party tenants', 'annual maintenance fee from 3rd party tenants',\
        'other services fee from 3rd party tenants', 'total hosting fee & services from 3rd party tenants',\
        'no of 3rd party tenants (pop count)', 'security class']
for i in ints:
    new[i] = new[i].fillna(0)
    new[i] = list(map(int, new[i]))

bill_col = lower_str(["Code","Categorization by Transmission Sys","Categorization by Transmission Sys (sub-cluster)",\
             "Categorization by Site Type",\
             "Categorization by Strategic Site","Categorization by Critical Site","Site Cluster",\
             "Strategic Site Bucket","Bts_sites","CriticalSite_Beyond_10","Sites As Metered Estimated","Active Sharing Arrangement",\
             "Indoor_Site_Any_Climate_Control","Outdoor_Site_With_Dc_Power","Rfai_Date","decommissioned site",\
             "Subsequent_Active_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Arrangement_Start_Date",\
             "First_Active_Sharing_Arrangement_End_Date"])

for i in bill_col:
    new[i] = new[i].replace(['N/A', 'n/a',"0", 0, '-'], '')

pathold = '/content/TowerDB_Germany_20210731.csv'
to_save = '/content/DE_TW'
old_n = '20210731.csv'
new_n = 'Release6.3.xlsx' 

find_diffs_between_files(pathold, new, 'Code', bill_col, 'Site Status (TIMS)', to_save,old_n, new_n, 'else','tw')

UIS Comparison

In [4]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix', dates_cols=[], status_col='', kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    import re
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule

    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        kind = kind.lower()
        if kind == 'ta':
            df = df.dropna(subset=[index_col], axis=0)
            df['sites'] = df[index_col].astype(str) + df[kind_col] 
            #df.columns = lower_str(list(df.columns))
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    def date_parser(df, columns, format=1, type_dates='normal'):
        t_col = type_dates.lower()
        if format == 1:
            type_date = "%d/%m/%Y"
        else:
            type_date = "%d-%m-%Y"
        for column in lower_str(columns):
            if t_col == 'mixed':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,engine='python').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, engine='python').fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        #cols_old = csv_header(path_OLD) header=0, names=cols_old,
        df_OLD = pd.read_csv(path_OLD, engine='python').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        date_parser(df_NEW, dates_cols, 1, 'mixed')
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ25000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ25000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

uis_inmonth = '/content/UserInput_Germany_20210831_InMonth.xlsx'
uis_old = '/content/UserInput_Germany_20210731_InMonth.xlsx'
uis_trueup = '/content/UserInput_Germany_20210630_TrueUps.xlsx'
sheet = 'SiteLevel'
uis_index = 'Site_ID (Alphanumeric)'
to_uis = '/content/DE_UIS_SiteLevel'
old_uis = '20210731.xlsx'
new_uis = '20210831_InMonth.xlsx'
bill = []
find_diffs_between_files(uis_trueup, uis_inmonth, uis_index, bill, to_uis, old_uis,new_uis,'excel',dates_cols=[],status_col='',\
                         kind='',kind_col='', sheetname=sheet, skipr=2, skipc=0)


New Rows:     ['0818 W', '0859 D', '1248 W', '1289 H', '2788 S', '4983 W', '5791 O', '7860 S', 'B003 M', 'MRT265']
Dropped Rows: ['0644 S', '1130 M', '1620 S', '2890 W', '2965 S', '3394 S', '3597 S', '4512 S', '5478 S', '5660 S', '7117 S', '7299 H', 'D074 W', 'MRT10', 'MRT112', 'MRT126', 'MRT147']

Done.



# TA Comparisons

In [2]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule
    import re
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        if kind == 'ta':
            df['sites'] = df[index_col] + df[kind_col]
            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        """if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()"""

        return newRows, droppedRows, new_copy
        """if kind=='ta':
            #new_sites['site_code'] = [i.endswith('MEO') or i.endswith('NOS')]
            new_copy['sites_code'] = [re.sub('MEO$|NOS$', '', str(x)) for x in new_copy['sites']]
            new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]"""

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, encoding='latin')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

ta_bills = ['Code','Tenant','Starting date', 'Expiring date']
path_ta_new = '/content/Output - Skylon - TowerDB - Germany - Release6.3 20210726_Billing_ReducedScope_sharedwithCelfinet_v03 (1).xlsx'
path_ta_old = '/content/TA_Input_Germany_20210731.csv'
ta_save = '/content/TA_DE'
old = '20210630.csv'
new = '20210731.csv'
"""(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0)"""
find_diffs_between_files(path_ta_old, path_ta_new, 'Tenant Agreement', ta_bills, ta_save, old, new,type_file='csv', status_col='',\
                         kind='ta', kind_col='Tenant')


New Rows:     []
Dropped Rows: ['/2500006069Telefónica Germany GmbH & Co. OHG', '2400003988/2500006284Landratsamt Niederschlesischer', '/2500006914EWE NETZ GmbH', '/2400000811Deutsche Funkturm GmbH', '/2400000810Deutsche Funkturm GmbH', '/2400000812Telekom Deutschland']

Done.



LC Input

In [ ]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        if kind == 'ta':
            df['sites'] = df[index_col] + df[kind_col]
            #fit_cols = lower_str(list(df.columns))
            #df.columns = fit_cols
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            df = df.dropna(subset=[index_col], axis=0)
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        """if kind=='tw':
            sites = [i for i in new_copy['sites']] 
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()"""

        return newRows, droppedRows, new_copy
        """if kind=='ta':
            #new_sites['site_code'] = [i.endswith('MEO') or i.endswith('NOS')]
            new_copy['sites_code'] = [re.sub('MEO$|NOS$', '', str(x)) for x in new_copy['sites']]
            new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]"""

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW, encoding='latin')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = fit_df(df_NEW, index_col,kind, kind_col)

    else:
        cols_old = csv_header(path_OLD)
        df_OLD = pd.read_csv(path_OLD,header=0, names=cols_old, engine='python',encoding='latin1').fillna('')
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

lc_bills = ['Lease Contract ID',' Annual lease fees ','Starting date', 'Expiring date']
path_lc_new = '/content/LC_Input_Germany_20210731.csv'
path_lc_old = '/content/LC_Input_Germany_20210630.csv'
lc_save = '/content/LC_DE'
old = '20210630.csv'
new = '20210731.csv'
"""(path_OLD, path_NEW, index_col, bill_cols, path_save, old_name,new_name, type_file='mix', status_col='',
                             kind='tw',kind_col='', sheetname='', skipr=0, skipc=0)"""
find_diffs_between_files(path_lc_old, path_lc_new, 'Lease Contract ID', lc_bills, lc_save, old, new,type_file='csv', status_col='',\
                         kind='ta', kind_col='Counterpart')